# Results p. 3: Boundary Sensitivity
## Saccade Boundaries

In [1]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


### Set Constants

In [2]:
THRESHOLD = 5       # temporal threshold for analyzing d'
METRIC = peyes.constants.D_PRIME_STR        # can also be `peyes.constants.F1_STR` or `peyes.constants.CRITERION_STR`
EVENT_NAME, EVENT_LABEL = "Saccade", 2    # EventLabelEnum.Saccade.value

GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH
LINE_WIDTH, ERROR_WIDTH = 2 * ZERO_LINE_WIDTH, ZERO_LINE_WIDTH
MARKER_SIZE = LINE_WIDTH * 2

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_TITLE_STANDOFF = 2

## Load Data

In [3]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

## Multi-Threshold Figure
### (not in the paper)
This figure shows the `METRIC` values at increasing temporal thresholds: $$\Delta t = 0, 1, ..., 20$$
Each detector is shown in a different color, with error bars showing the SEM across recordings.
The left colomn show results when `GT1` (_RA_) is used as the ground truth, while the right column shows results when `GT2` (_MN_) is used.

In [4]:
W, H = 1300, 450

multi_thresholds_figure = ch_sdt.multi_channel_figure(
    csdt_metrics,
    metric=METRIC,
    yaxis_title=r"d'", show_other_gt=True,
    error_bars='std', show_err_bands=False,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_hspace=0, subplots_vspace=0,
)

multi_thresholds_figure.update_layout(
    width=W, height=H,
    title=dict(text=EVENT_NAME + "s", y=0.975, x=0.5, xanchor='center'),
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',

    # remove axis grids
    xaxis=dict(showgrid=False, zeroline=False, showline=False), yaxis=dict(showgrid=False, zeroline=False, showline=False),
    xaxis2=dict(showgrid=False, zeroline=False, showline=False), yaxis2=dict(showgrid=False, zeroline=False, showline=False),
    xaxis3=dict(showgrid=False, zeroline=False, showline=False), yaxis3=dict(showgrid=False, zeroline=False, showline=False),
    xaxis4=dict(showgrid=False, zeroline=False, showline=False), yaxis4=dict(showgrid=False, zeroline=False, showline=False),

    # move legend to bottom
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.05),
    # showlegend=False,   # hide legend
    margin=dict(l=10, r=0, b=10, t=0, pad=0),
)
multi_thresholds_figure.layout.annotations = []   # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 6, '', True
# save_fig(multi_thresholds_figure, FIG_ID, PANEL_ID, f"{TITLE.lower()}discrimination_multi_threshold-{METRIC.lower()}", IS_SUPP)

# multi_thresholds_figure.show()

## Statistical Analysis
### Onsets

In [5]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc = ch_sdt.post_hoc_table(
    onset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([onset_statistics, onset_pvalues, onset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  66.449032  2.181681e-12    True
                 RA  93.029810  7.106495e-18    True
d_prime          MN  64.095484  6.599235e-12    True
                 RA  86.926829  1.316055e-16    True
f1               MN  64.823226  4.687800e-12    True
                 RA  86.298103  1.776808e-16    True
false_alarm_rate MN  42.975207  1.179687e-07    True
                 RA  59.283495  6.292337e-11    True
precision        MN  39.197889  6.545891e-07    True
                 RA  45.465181  3.782614e-08    True
recall           MN  65.706122  3.094863e-12    True
                 RA  93.100671  6.869345e-18    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.      **     n.s.
         RA      --    n.s.      **      **    n.s.     ***     n.s.
ivvt     MN  1.0000      --       *       *    n.s.      **     n.s.
         RA  0.9999      --      **       *    n.s.     ***     n.s.
idt      MN  0.0179  0.0205      --    n.s.      **    n.s.      ***
         RA  0.0010  0.0049      --    n.s.     ***    n.s.      ***
idvt     MN  0.0285  0.0325  1.0000      --      **    n.s.      ***
         RA  0.0025  0.0108  1.0000      --     ***    n.s.      ***
engbert  MN  0.9994  0.9991  0.0027  0.0048      --     ***     n.s.
         RA  0.9952  0.9567  0.0000  0.0001      --     ***     n.s.
nh       MN  0.0020  0.0024  0.9987  0.9956  0.0002      --      ***
         RA  0.0001  0.0006  0.9996  0.9965  0.0000      --      ***
remodnav MN  0.9704  0.9639  0.0003  0.0005  0.9993  0.0000       --
         RA  0.9870  0.9220  0.0000  0.0000  1.0000  0.0000       --

### Offsets

In [6]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc = ch_sdt.post_hoc_table(
    offset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([offset_statistics, offset_pvalues, offset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  62.748387  1.242061e-11    True
                 RA  86.755877  1.427991e-16    True
d_prime          MN  31.981935  1.644822e-05    True
                 RA  73.735986  6.986369e-14    True
f1               MN  30.948682  2.592760e-05    True
                 RA  60.879518  2.982278e-11    True
false_alarm_rate MN  37.769437  1.246169e-06    True
                 RA  45.397388  3.901880e-08    True
precision        MN  34.459355  5.483767e-06    True
                 RA  57.377376  1.532165e-10    True
recall           MN  52.937913  1.207557e-09    True
                 RA  88.722581  5.580694e-17    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.      **      **    n.s.    n.s.     n.s.
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.1478  0.0762      --    n.s.    n.s.    n.s.      ***
         RA  0.0014  0.0005      --    n.s.    n.s.       *      ***
idvt     MN  0.1623  0.0850  1.0000      --    n.s.    n.s.      ***
         RA  0.0011  0.0004  1.0000      --    n.s.       *      ***
engbert  MN  0.8610  0.7288  0.9017  0.9149      --    n.s.        *
         RA  0.7340  0.5919  0.2648  0.2381      --    n.s.        †
nh       MN  1.0000  1.0000  0.1688  0.1847  0.8841      --     n.s.
         RA  0.9926  0.9711  0.0271  0.0226  0.9850      --     n.s.
remodnav MN  0.5759  0.7389  0.0001  0.0001  0.0264  0.5378       --
         RA  0.8806  0.9489  0.0000  0.0000  0.0664  0.4346       --

## Figure: Distribution @ Threshold
### (not in the paper)
This figure shows the distribution of `METRIC` at the predefined temporal `THRESHOLD`, for each detector across recordings.
The right-side of each violin is w.r.t. `GT1` (_RA_) as the ground truth annotator, and the left-side is w.r.t. `GT2` (_MN_).

In [7]:
W, H = 600, 450

onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

######################################
## COMBINE ONSET AND OFFSET FIGURES ##
######################################

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=[f"{EVENT_NAME} {peyes.constants.ONSET_STR}", f"{EVENT_NAME} {peyes.constants.OFFSET_STR}"]
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles

# FIG_ID, IS_SUPP = 5, False
# save_fig(discriminability_figure, FIG_ID, 'left', 'saccade-discriminability', IS_SUPP)

# discriminability_figure.show()

## Final Figure
### (Shown in the paper)
- Combine Multi-Threshold line-plots with Single-Threshold violin-plots
- use only one GT annotator

In [8]:
WIDTH, HEIGHT = 1600, 900
COLUMN_TITLES = [f"{EVENT_NAME} {peyes.constants.ONSET_STR.capitalize()}s", f"{EVENT_NAME} {peyes.constants.OFFSET_STR.capitalize()}s",]


def _convert_line_traces(line_fig: go.Figure, new_fig: go.Figure, gt: str) -> go.Figure:
    if gt == "RA":
        other = "MN"
        allowed_yaxes = ["y", "y3"]
    elif gt == "MN":
        other = "RA"
        allowed_yaxes = ["y2", "y4"]
    else:
        raise ValueError("Unknown GT: {}".format(gt))

    for trace in line_fig.data:
        if not trace['yaxis'] in allowed_yaxes:
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr["showlegend"] = False
        new_tr["line_width"] = LINE_WIDTH
        new_tr["error_y"]["thickness"] = ERROR_WIDTH
        new_tr["marker"]["size"] = LINE_WIDTH * 2

        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
            new_tr["marker"]["color"] = new_tr["line"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] in ['y', 'y2']   # onset traces are in top row
        new_fig.add_trace(new_tr, row=1, col=1 if is_onset else 2)
    return new_fig


def _convert_violin_traces(violin_fig, new_fig, gt: str) -> go.Figure:
    other = "MN" if gt == "RA" else "RA"
    for trace in violin_fig.data:
        if trace['scalegroup'] != METRIC:
            # ignore non `METRIC` traces
            continue
        if not trace["name"].startswith(gt):
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr['scalegroup'] = "VIOLINS"
        new_tr["showlegend"] = False

        # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
        if new_tr["x0"] == "Other GT":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
        elif new_tr["x0"].startswith("i"):
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
        elif new_tr["x0"] == "remodnav":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
        new_tr['x'] = new_tr['y']

        # reset irrelevant props
        new_tr['y'] = new_tr['x0'] = None
        new_tr["showlegend"] = new_tr['points'] = False
        new_tr["side"] = "positive"
        new_tr["width"] = 1.8
        new_tr["box"] = dict(visible=False, width=0.95, line=dict(width=1))
        new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
        new_tr["opacity"] = 1
        new_tr["visible"] = True

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] == 'y'
        new_fig.add_trace(new_tr, row=2, col=1 if is_onset else 2)
    return new_fig


def convert_figs(line_fig, violin_fig, gt: str) -> go.Figure:
    # copy traces to new figure
    new_fig = make_subplots(
        rows=len(COLUMN_TITLES), cols=len(COLUMN_TITLES),
        shared_xaxes=False, shared_yaxes=True,
        vertical_spacing=0.12, horizontal_spacing=0.02,
        column_titles=COLUMN_TITLES,
    )
    _convert_line_traces(line_fig, new_fig, gt)
    _convert_violin_traces(violin_fig, new_fig, gt)

    # update legends
    new_fig.for_each_trace(lambda tr: tr.update(showlegend=tr['yaxis'] == 'y'))  # legend only shows one set of lines

    # update subtitles
    new_fig.for_each_annotation(lambda ann: ann.update(text=f"<b>{ann.text}</b>", font=TITLE_FONT, textangle=0, ))

    # add red rectangles in the top plots to to highlight the bottom subplots' origin
    for c in range(len(COLUMN_TITLES)):
        if gt=="RA":
            y0 = 0.8 if c == 0 else -0.35
            y1 = 5.9 if c == 0 else 5.2
        else:
            y0 = 0.9 if c == 0 else -0.15
            y1 = 5.7 if c == 0 else 4.8
        new_fig.add_shape(
            row=1, col=c + 1,
            type="rect",
            x0=THRESHOLD-0.5, x1=THRESHOLD+0.5,
            y0=y0, y1=y1,
            line_color="red", line_width=LINE_WIDTH,
        )

    # add axis labels
    for r in range(len(COLUMN_TITLES)):
        xaxis_title = "Δt (samples)" if r == 0 else r"$d'$"
        yaxis_title = r"$d'$" if r == 0 else "Detector"
        for c in range(len(COLUMN_TITLES)):
            new_fig.update_xaxes(
                row=r + 1, col=c + 1,
                title=dict(text=xaxis_title, font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=r == 0, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                tickfont=AXIS_TICK_FONT,
            )
            new_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=yaxis_title if c == 0 else "", font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                showticklabels=r==0 and c==0, tickfont=AXIS_TICK_FONT, tickangle=0 if r==0 else 30,
            )

    # add row annotations "A" and "B"
    for ann in ["A", "B"]:
        new_fig.add_annotation(
            text=f"<b>{ann}</b>", font={**TITLE_FONT, **dict(size=36)}, showarrow=False,
            xref="paper", yref="paper", xanchor="right", yanchor="top", x=0, y=1.0525 if ann == "A" else 0.46
        )


    new_fig.update_layout(
        font_family=FONT_FAMILY,
        width=WIDTH, height=HEIGHT,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            orientation="h", bgcolor='rgba(0, 0, 0, 0)',
            yanchor="top", y=-0.06, xanchor="center", x=0.5,
            font=AXIS_TICK_FONT, itemwidth=90,
        ),
    )
    return new_fig

In [9]:
GT = "RA"
NAME = f"fig5-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

In [10]:
GT = "MN"
NAME = f"supp_fig_H1-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

In [11]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics
csdt_subset = csdt_metrics.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)

### Onsets

In [12]:
onset_statistics_subset, onset_pvalues_subset, onset_nemenyi_subset, onset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc_subset = ch_sdt.post_hoc_table(
    onset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [onset_statistics_subset, onset_pvalues_subset, onset_pvalues_subset <= ALPHA],
        axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  61.416541  2.319073e-11    True
                 RA  80.178129  3.282617e-15    True
d_prime          MN  58.745865  8.089311e-11    True
                 RA  78.284104  8.077211e-15    True
f1               MN  58.619549  8.580935e-11    True
                 RA  78.635851  6.834006e-15    True
false_alarm_rate MN  38.027778  1.109402e-06    True
                 RA  56.704225  2.096942e-10    True
precision        MN  35.698630  3.154443e-06    True
                 RA  48.346420  1.007489e-08    True
recall           MN  60.412141  3.711626e-11    True
                 RA  80.101449  3.404557e-15    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.     ***     n.s.
         RA      --    n.s.      **       *    n.s.     ***     n.s.
ivvt     MN  1.0000      --       †    n.s.    n.s.     ***     n.s.
         RA  0.9988      --       *    n.s.    n.s.     ***     n.s.
idt      MN  0.0383  0.0557      --    n.s.       *    n.s.       **
         RA  0.0071  0.0472      --    n.s.      **    n.s.      ***
idvt     MN  0.0632  0.0889  1.0000      --       *    n.s.        *
         RA  0.0148  0.0832  1.0000      --      **    n.s.       **
engbert  MN  1.0000  0.9999  0.0161  0.0283      --     ***     n.s.
         RA  0.9997  0.9723  0.0011  0.0027      --     ***     n.s.
nh       MN  0.0005  0.0009  0.9543  0.9106  0.0001      --      ***
         RA  0.0000  0.0003  0.8963  0.8116  0.0000      --      ***
remodnav MN  0.9991  0.9971  0.0060  0.0114  1.0000  0.0000       --
         RA  0.9995  0.9661  0.0009  0.0023  1.0000  0.0000       --

### Offsets

In [13]:
offset_statistics_subset, offset_pvalues_subset, offset_nemenyi_subset, offset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc_subset = ch_sdt.post_hoc_table(
    offset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [offset_statistics_subset, offset_pvalues_subset, offset_pvalues_subset <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  53.927820  7.628578e-10    True
                 RA  75.304740  3.321974e-14    True
d_prime          MN  26.733835  1.624119e-04    True
                 RA  62.424379  1.445930e-11    True
f1               MN  24.736498  3.820148e-04    True
                 RA  49.005155  7.438259e-09    True
false_alarm_rate MN  34.018868  6.671197e-06    True
                 RA  46.597222  2.251177e-08    True
precision        MN  36.009023  2.745518e-06    True
                 RA  63.450847  8.932426e-12    True
recall           MN  42.175385  1.697820e-07    True
                 RA  70.600000  3.079764e-13    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.     ***     ***    n.s.    n.s.     n.s.
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.1126  0.0752      --    n.s.    n.s.    n.s.      ***
         RA  0.0007  0.0003      --    n.s.    n.s.       *      ***
idvt     MN  0.1233  0.0830  1.0000      --    n.s.    n.s.      ***
         RA  0.0005  0.0002  1.0000      --    n.s.       *      ***
engbert  MN  0.9377  0.8907  0.7442  0.7647      --    n.s.     n.s.
         RA  0.6983  0.6022  0.2207  0.1899      --    n.s.     n.s.
nh       MN  1.0000  1.0000  0.0969  0.1065  0.9218      --     n.s.
         RA  0.9906  0.9767  0.0182  0.0142  0.9826      --     n.s.
remodnav MN  0.8561  0.9130  0.0007  0.0008  0.1895  0.8798       --
         RA  0.9767  0.9906  0.0000  0.0000  0.1552  0.6661       --